In [ ]:
from UtilityLib import ProjectManager
T0104 = ProjectManager(path_base=r"/mnt/DataDrive/MDD/T0104--Abhimanyu-GeneExpression/TCGA-BRCA-Analysis-of-Genes-AN/")


In [ ]:
_sample_sheet = T0104.pd_tsv(T0104.find_files(T0104.get_path(), "gdc_manifest*")[0])
_sample_sheet.columns = [_col.replace(' ', '_') for _col in _sample_sheet.columns]
T0104.pd_excel(T0104.get_path("sample-data.xlsx"), _sample_sheet, "Samples")
_sample_sheet.head(2)

In [ ]:
"""For star count"""
if not isinstance(T0104.config.exp.star_files, (list, tuple)):
    T0104.config.exp.star_files = []

def convert_file_set_to_df(T0104, _file_set, _chunk_path):
    _expression_df = None
    for _f in _file_set:
        _fn = T0104.filename(_f, with_ext=True)
        if _f in T0104.config.exp.star_files:
            continue
        _df = T0104.pd_tsv(_f, skiprows=1)
        _df['FileName'] = _fn
        _expression_df = _df if _expression_df is None else T0104.PD.concat([_expression_df, _df], axis=0)
        T0104.config.exp.star_files.append(_fn)
    _expression_df.to_pickle(_chunk_path)


_expression_path = T0104.get_path('expression-star-count.pkl.gz')
if T0104.exists(_expression_path):
    _combined_df = T0104.unpickle(_expression_path)
else:
    print("Reprocessing Files...")
    T0104.validate_dir(T0104.get_path('Read-Chunks'))
    T0104.init_multiprocessing()
    _chunk_size = 20
    _expression_files = T0104.find_files(T0104.get_path("TCGA-Downloads"), "*/*tsv")
    _chunk_list = list(T0104.chunks(_expression_files, _chunk_size))
    for _chunk_idx, _file_set in T0104.PB(enumerate(_chunk_list), total=len(_chunk_list)):
        _chunk_path = T0104.get_path(f'Read-Chunks/File-Chunk-{_chunk_idx}.pkl.gz')
        if T0104.exists(_chunk_path):
            continue
        # convert_file_set_to_df(T0104, _file_set, _chunk_path)
        T0104.queue_task(convert_file_set_to_df, T0104, _file_set, _chunk_path)
        T0104.update_config()

    T0104.process_queue()
    """Combined all the read files"""
    _pkl_dfs = T0104.find_files(T0104.get_path("Read-Chunks"), "*pkl.gz")
    _pkl_dfs_reads = []
    for _pkf in T0104.PB(_pkl_dfs):
        _pkl_dfs_reads.append(T0104.unpickle(_pkf))

    _combined_df = T0104.PD.concat(_pkl_dfs_reads, axis=0)

    # _combined_df = None
    # for _pkf in T0104.PB(_pkl_dfs):
    #     _df = T0104.unpickle(_pkf)
    #     _combined_df = _df if _combined_df is None else T0104.PD.concat([_combined_df, _df], axis=0)

    _combined_df.to_pickle(_expression_path)
    print(_combined_df.shape) # ~ (96760675, 2)

print(_combined_df.head(2))

In [ ]:
_combined_df.head(2)

In [ ]:
_filter_exp = _combined_df.gene_id.str.startswith('N_')

_mapping_data = _combined_df[_filter_exp].copy()
_expression_data = _combined_df[~_filter_exp].copy()

_mapping_data.shape, _expression_data.shape